## Kütüphaneler import edildi

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt

## Classlar bulundu

In [ ]:
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset' 
classes = [i for i in os.listdir(fish_dir) if '.' not in i]                    
classes

## Veriler dizilere işlendi

In [ ]:
label = []
path = []

for dir_name, _,filenames in os.walk(fish_dir):                    
    for filename in filenames:                                 
        if os.path.splitext(filename)[-1]=='.png':               # If filename contains .png
            if dir_name.split()[-1]!='GT':                       # If directory doesn't contain GT
                label.append(os.path.split(dir_name)[-1])         # Append the directory name to label 
                path.append(os.path.join(dir_name,filename))     # Append all the png files to path of that directory

data = pd.DataFrame(columns=['path','label'])
data['path']=path
data['label']=label

## Train ve Validation setler hazırlandı
Siyah beyaz resimler de kullanıldı.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=fish_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    subset='training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    directory=fish_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    subset='validation',
    class_mode='categorical')

## Model ve katmanlar oluşturuldu
Dropout uyguladığımda accuracy çok düşüyordu. Bu yüzden uygulamadım.

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(224, 224, 3)),
    layers.Flatten(),  # Flatten the 3D image to 1D vector
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# Print model summary
model.summary()

## Model Eğitildi

In [ ]:
history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

## Doğruluk Metrikleri

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: %{loss*100}')
print(f'Validation accuracy: %{accuracy*100}')

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.show()

In [ ]:
 from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Test verisini almak için validation_generator'ı kullanın
y_pred = model.predict(validation_generator)  # Validasyon seti için tahmin yap
y_pred_classes = np.argmax(y_pred, axis=1)  # Tahmin edilen sınıf indekslerini al

# Gerçek etiketler
y_true = validation_generator.classes  # Validation setinin gerçek etiketleri
class_labels = list(validation_generator.class_indices.keys())  # Sınıf etiketlerini al

# Sınıflandırma raporu
report = classification_report(y_true, y_pred_classes, target_names=class_labels)
print(report)

# Karmaşıklık matrisi
cm = confusion_matrix(y_true, y_pred_classes)

# Karmaşıklık matrisini görselleştir
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('Gerçek Sınıf')
plt.xlabel('Tahmin Edilen Sınıf')
plt.title('Karmaşıklık Matrisi')
plt.show()
